# 3D Droplet Oscillation (Part 2, Postprocessing)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
//BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D_FirstPeriodStudy");

Project name is set to 'OscillatingDroplet3D_FirstPeriodStudy'.
Opening existing database 'D:\local\OscillatingDroplet3D_FirstPeriodStudy'.


In [ ]:
//add database manually:
OpenOrCreateDatabase(@"\\dc1\userspace\smuda\Databases\OscillatingDroplet3D");
//OpenOrCreateDatabase(@"\\130.83.248.207\ValidationTests\OscillatingDroplet3D_FirstPeriodStudy");

Opening existing database '\\dc1\userspace\smuda\Databases\OscillatingDroplet3D'.


## Observing sessions

In [ ]:
databases.Pick(0).Sessions

#0: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta05	04/08/2022 11:32:02	9173b638...
#1: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta06	04/08/2022 11:31:47	6feb68ca...
#2: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta02	04/08/2022 11:32:41	012ffd75...
#3: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta03	04/08/2022 11:32:24	1de3f59e...
#4: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta07	04/08/2022 11:31:37	de146e9e...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005*	04/08/2022 11:37:03	92930af7...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01*	04/08/2022 11:36:37	15341e01...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04*	04/08/2022 11:34:56	8486c026...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04*	04/08/2022 11:34:00	e3a04ed7...
#9: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh

In [ ]:
// var studySess = wmg.Sessions.Skip(6);
// studySess

In [ ]:
//databases

In [ ]:
//var targetDB = databases.Pick(3);
//targetDB
//databases.Pick(2).Sessions.MoveAll(targetDB);
//wmg.Sessions[0].Move(targetDB);
//wmg.Sessions.Where(s => s.Name.Contains("J432k3_arm0_case2_Oh0.1_AnalytInit"))
//wmg.Sessions[0].Delete(true)
//wmg.Sessions[8].Timesteps.Count()

In [ ]:
int sNum = 7;
wmg.Sessions[sNum]

OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta07*	04/08/2022 11:30:52	714bd2c2...

In [ ]:
wmg.Sessions[sNum].GetSessionDirectory()

\\130.83.248.207\ValidationTests\OscillatingDroplet3D\sessions\23a6e85a-e66a-45d8-8b8b-919a6cc3de91

In [ ]:
wmg.Sessions[sNum].DeployPath

\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2022Feb21_132417

In [ ]:
wmg.Sessions[sNum].Timesteps.Count

212

In [ ]:
wmg.Sessions[sNum].Timesteps.Export().WithSupersampling(2).Do();

Starting export process... Data will be written to the directory: C:\Users\smuda\AppData\Local\BoSSS\plots\sessions\OscillatingDroplet3D_FirstPeriodStudy__OD3D_J686k3_wallBC_amr1_m3_Oh01_eta07__714bd2c2-f2bd-49fe-88ba-905f7c4e7825


In [ ]:
using System.IO;

# Study Overview

In [ ]:
var studySess = databases.Pick(0).Sessions.Take(26);
studySess

#0: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta05	04/08/2022 11:32:02	9173b638...
#1: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta06	04/08/2022 11:31:47	6feb68ca...
#2: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta02	04/08/2022 11:32:41	012ffd75...
#3: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta03	04/08/2022 11:32:24	1de3f59e...
#4: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta07	04/08/2022 11:31:37	de146e9e...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005*	04/08/2022 11:37:03	92930af7...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01*	04/08/2022 11:36:37	15341e01...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04*	04/08/2022 11:34:56	8486c026...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04*	04/08/2022 11:34:00	e3a04ed7...
#9: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh

In [ ]:
string[] caseS = new string[] { "m2_Oh01_eta07", "m2_Oh01_eta06", "m2_Oh01_eta05", "m2_Oh01_eta03", "m3_Oh01_eta07", "m3_Oh01_eta06", "m3_Oh01_eta05", "m4_Oh01_eta07", "m4_Oh01_eta06", "m4_Oh01_eta05", "m4_Oh01_eta03", "m4_Oh01_eta02"};
string[] gridS = new string[] { "J686" };
bool[] ShouldAnaInit = new bool[] { false };
bool[] useNewton = new bool[] { false };

# Plotting  droplet metrics - aspect ratio L/W

In [ ]:
//string[] metricKeys = new string[] {"theta0", "theta90x", "theta90y", "volume"};

In [ ]:
Plot2Ddata[,] PlotTableDM = new Plot2Ddata[caseS.Length,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
for(int iGrd = 0; iGrd < gridS.Length; iGrd++) {   
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {

    string _case = caseS[iRow];
    string _grd = gridS[iGrd];

    // if(_case == "case2" || _case == "case3")
    //     continue; 

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_grd)
                                      && (sess.Name.Contains("_thirdOrderInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));


    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    if(SIs.IsNullOrEmpty()) 
        continue;

    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("DropletMetrics.txt", '\t');
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        // var dummyPlot = new Plot2Ddata();
        // dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        // PlotTableDM[iRow,iCol] = dummyPlot;
        continue;
    }
    
    var plot = new Plot2Ddata();
    var time = tab["time"];

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    if (useNewton[iNew]) {
        fmt.LineColor = LineColors.Red;
        //fmt.DashType = DashTypes.Dashed;
    } else {  
        fmt.LineColor = LineColors.Blue;
        //fmt.DashType = DashTypes.Solid;
    }
    if (!ShouldAnaInit[iAna]) {
        fmt.DashType = DashTypes.Solid;
    } else {
        fmt.DashType = DashTypes.Dashed;
    }
    
    // if (_init == "_initCase0")
    //     fmt.LineColor = LineColors.Green;
    // if (_init == "_initCase1")
    //     fmt.LineColor = LineColors.Magenta;
    // if (_init == "_initCase2")
    //     fmt.LineColor = LineColors.Orange;
 
    string name = "BoSSS-"+_grd+_case;
    // if(iAMR > 0)
    //     name = name + "-amr";
    if(ShouldAnaInit[iAna])
        name = name + "-3OrdInit"; 
    if(useNewton[iNew])
        name = name + "-Newton";

    double[] L = tab["L"].ToArray(); 
    double[] Wx = tab["Wx"].ToArray(); 
    double[] Wy = tab["Wy"].ToArray();
    double[] arLW = new double[L.Length];
    for (int i = 0; i < L.Length; i++) {
        arLW[i] = 0.5 * ((L[i] / (2.0*Wx[i])) + (L[i] / (2.0*Wy[i])));
    }
    
    // List<double> arLW_lst = new List<double>();
    // List<double> time_lst = new List<double>();
    // double dataThreshold = 0.1;
    // for(int i = 0; i < L.Length; i++) {
    //     if(arLW[i] > dataThreshold) {
    //         arLW_lst.Add(arLW[i]);
    //         time_lst.Add(time[i]);
    //     }
    // }
    // arLW = arLW_lst.ToArray();
    // time = time_lst.ToArray();

    plot.AddDataGroup(name, time, arLW, fmt);
   
    plot.ShowLegend = true;  

    if(PlotTableDM[iRow,iCol]==null) {
        PlotTableDM[iRow,iCol] = plot;
    } else {
        PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    }
        
    // if(iAMR == 0) {
    //     PlotTableDM[iRow,iCol] = plot;
    // } else {
    //     PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    // }
    
}

if(PlotTableDM[iRow,iCol] != null) {
    PlotTableDM[iRow,iCol].ShowLegend = true; 
    //PlotTableDM[iRow,iCol].LegendFont = 10;
    if(iCol == 0) {
       PlotTableDM[iRow,iCol].Title = caseS[iRow];
       PlotTableDM[iRow,iCol].Xlabel = "time t";
       PlotTableDM[iRow,iCol].Ylabel = "aspect ratio L/W";
    }
}
}
}
}
}

OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta03
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta07
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta06
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta05
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta07
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta06
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta05
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta03
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta02


In [ ]:
PlotTableDM

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> <null> 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> <null> 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> <null> 3 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> <null> 4 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> <null> 5 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> <null> 6 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> <null> 7 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> <null> 8 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> <null> 9 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> <null> 10 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> <null> 11 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> <null>

## m=2 cases (eta = {0.7, 0.6, 0.5, 0.3})

In [ ]:
var pltTab1 = PlotTableDM[0,0];
PlotTableDM[1,0].dataGroups[0].Format.LineColor = LineColors.Magenta;
pltTab1 = pltTab1.Merge(PlotTableDM[1,0]);
PlotTableDM[2,0].dataGroups[0].Format.LineColor = LineColors.Green;
pltTab1 = pltTab1.Merge(PlotTableDM[2,0]);
PlotTableDM[3,0].dataGroups[0].Format.LineColor = LineColors.Orange;
pltTab1 = pltTab1.Merge(PlotTableDM[3,0]);

In [ ]:
pltTab1.XrangeMin = 0;
pltTab1.XrangeMax = 2;
var gp = pltTab1.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 BoSSS-J686m2Oh01eta07 
 
 
 
 
 BoSSS-J686m2 O h01 e ta07 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M455.3,57.1 L508.7,57.1 M53.9,49.8 L55.6,49.8 L57.4,50.5 L59.1,51.7 L60.9,53.3 L62.6,55.3
 L64.3,57.5 L66.1,59.8 L67.8,62.3 L69.6,64.8 L71.3,67.2 L73.0,69.7 L74.8,72.0 L76.5,74.3
 L78.3,76.5 L80.0,78.8 L81.7,80.9 L83.5,83.1 L85.2,85.2 L87.0,87.3 L88.7,89.4 L90.5,91.5
 L92.2,93.6 L93.9,95.7 L95.7,97.8 L97.4,99.9 L99.2,102.0 L100.9,104.1 L102.6,106.2 L104.4,108.3
 L106.1,110.3 L107.9,112.4 L109.6,116.5 L111.3,118.3 L113.1,121.4 L114.8,123.1 L116.6,124.9 L118.3,126.7
 L120.0,128.3 L121.8,129.9 L123.5,131.5 L125.3,133.3 L127.0,135.1 L128.7,136.9 L130.5,138.7 L132.2,140.5
 L134.0,142.8 L135.7,144.6 L137.4,146.3 L139.2,147.9 L140.9,149.4 L142.7,151.9 L144.4,153.4 L146.1,154.9
 L147.9,156.9 L149.6,158.3 L151.4,159.8 L153.1,161.2 L154.8,162.7 L156.6,164.1 L158.3,165.8 L160.1,167.5
 L161.8,169.1 L163.6,170.6 L165.3,172.0 L167.0,173.7 L168.8,175.2 L170.5,176.6 L172.3,178.2 L174.0,179.7
 L175.7,181.2 L177.5,182.6 L179.2,184.0 L181.0,185.4 L182.7,186.8 L184.4,188.2 L186.2,189.6 L187.9,191.0
 L189.7,192.3 L191.4,193.7 L193.1,195.0 L194.9,196.4 L196.6,197.7 L198.4,199.1 L200.1,200.2 L201.8,201.6
 L203.6,203.0 L205.3,204.3 L207.1,205.7 L208.8,207.0 L210.5,208.3 L212.3,209.4 L214.0,210.7 L215.8,212.1
 L217.5,213.4 L219.2,214.7 L221.0,216.0 L222.7,217.0 L224.5,218.3 L226.2,219.6 L228.0,221.0 L229.7,222.3
 L231.4,223.3 L233.2,224.6 L234.9,226.1 L236.7,227.5 L238.4,228.9 L240.1,230.2 L241.9,231.6 L243.6,232.9
 L245.4,234.2 L247.1,235.5 L248.8,236.8 L250.6,238.1 L252.3,239.4 L254.1,240.7 L255.8,242.1 L257.5,243.4
 L259.3,244.9 L261.0,246.2 L262.8,247.6 L264.5,248.9 L266.2,250.4 L268.0,251.8 L269.7,253.2 L271.5,254.6
 L273.2,256.0 L274.9,257.4 L276.7,258.9 L278.4,260.4 L280.2,261.9 L281.9,263.4 L283.6,264.9 L285.4,266.5
 L287.1,268.0 L288.9,270.5 L290.6,272.0 L292.3,273.4 L294.1,274.8 L295.8,276.4 L297.6,278.0 L299.3,279.6
 L301.1,281.3 L302.8,283.6 L304.5,285.1 L306.3,286.7 L308.0,288.6 L309.8,290.2 L311.5,291.8 L313.2,293.4
 L315.0,295.2 L316.7,296.9 L318.5,298.7 L320.2,300.5 L321.9,302.3 L323.7,304.1 L325.4,305.8 L327.2,307.7
 L328.9,309.5 L330.6,311.3 L332.4,313.2 L334.1,315.1 L335.9,316.9 L337.6,318.8 L339.3,320.4 L341.1,322.3
 L342.8,324.2 L344.6,325.8 L346.3,327.7 L348.0,329.5 L349.8,331.5 L351.5,333.4 L353.3,335.3 L355.0,337.0
 L356.7,339.0 L358.5,340.9 L360.2,342.7 L362.0,344.6 L363.7,346.5 L365.4,348.3 L367.2,350.2 L368.9,352.2
 L370.7,354.0 L372.4,355.8 L374.2,357.7 L375.9,359.6 L377.6,361.4 L379.4,363.2 L381.1,364.8 L382.9,366.6
 L384.6,368.4 L386.3,370.2 L388.1,372.0 L389.8,373.8 L391.6,375.5 L393.3,377.2 L395.0,378.9 L396.8,380.6
 L398.5,382.2 L400.3,383.9 L402.0,385.5 L403.7,387.2 L405.5,388.8 L407.2,390.4 L409.0,392.1 L410.7,393.7
 L412.4,395.3 L414.2,396.8 L415.9,398.4 L417.7,399.9 L419.4,401.4 L421.1,402.9 L422.9,404.5 L424.6,405.8
 L426.4,407.2 L428.1,408.6 L429.8,410.0 L431.6,411.4 L433.3,412.9 L435.1,414.1 L436.8,415.6 L438.6,416.8
 L440.3,418.0 L442.0,419.5 L443.8,421.1 L445.5,422.2 L447.3,423.5 L449.0,424.6 L450.7,425.4 L452.5,426.3
 L454.2,427.6 L456.0,428.4 L457.7,429.3 L459.4,430.3 L461.2,431.5 L462.9,432.3 L464.7,433.2 L466.4,434.2
 L468.1,435.1 L469.9,436.0 L471.6,437.0 L473.4,437.8 L475.1,438.6 L476.8,439.4 L478.6,440.2 L480.3,441.0
 L482.1,441.7 L483.8,442.4 L485.5,443.0 L487.3,443.7 L489.0,444.3 L490.8,445.0 L492.5,445.6 L494.2,446.2
 L496.0,446.8 L497.7,447.3 L499.5,447.8

In [ ]:
// foreach(var grp in pltTab1.dataGroups) {
//     Plot2Ddata dat = new Plot2Ddata();
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m2/" + pltTab1.Title + "/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

## m=3 cases (eta = {0.7, 0.6, 0.5})

In [ ]:
var pltTab2 = PlotTableDM[4,0];
PlotTableDM[5,0].dataGroups[0].Format.LineColor = LineColors.Magenta;
pltTab2 = pltTab2.Merge(PlotTableDM[5,0]);
PlotTableDM[6,0].dataGroups[0].Format.LineColor = LineColors.Green;
pltTab2 = pltTab2.Merge(PlotTableDM[6,0]);

In [ ]:
pltTab1.XrangeMin = 0;
pltTab1.XrangeMax = 2;
var gp = pltTab2.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 BoSSS-J686m3Oh01eta07 
 
 
 
 
 BoSSS-J686m3 O h01 e ta07 
 
 
 
 
 
 BoSSS-J686m3Oh01eta06 
 
 
 BoSSS-J686m3 O h01 e ta06 
 
 
 
 
 
 BoSSS-J686m3Oh01eta05 
 
 
 BoSSS-J686m3 O h01 e ta05 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M455.3,105.1 L508.7,105.1 M53.9,563.5 L56.8,563.5 L59.7,563.8 L62.6,563.9 L65.5,564.0 L68.4,564.0
 L71.3,179.4 L74.2,563.5 L77.1,563.5 L80.0,563.7 L82.9,564.0 L85.8,563.9 L88.7,564.0 L91.6,563.8
 L94.5,563.9 L97.4,563.9 L100.3,564.0 L103.2,563.9 L106.1,563.9 L109.0,155.0 L111.9,564.0 L114.8,564.0
 L117.7,150.1 L120.6,563.8 L123.5,563.9 L126.4,563.8 L129.3,563.9 L132.2,563.7 L135.1,563.7 L138.0,563.9
 L140.9,563.9 L143.8,563.9 L146.7,564.0 L149.6,564.0 L152.5,564.0 L155.4,563.9 L158.3,563.9 L161.2,563.9
 L164.1,131.7 L167.0,563.9 L169.9,563.9 L172.8,564.0 L175.7,126.3 L178.6,564.0 L181.5,561.8 L184.4,563.8
 L187.3,563.7 L190.2,560.7 L193.1,122.6 L196.0,121.1 L198.9,564.0 L201.8,564.0 L204.7,564.0 L207.6,564.0
 L210.5,564.0 L213.4,564.0 L216.3,564.0 L219.2,564.0 L222.1,111.9 L225.0,111.2 L228.0,564.0 L230.9,564.0
 L233.8,109.3 L236.7,564.0 L239.6,564.0 L242.5,564.0 L245.4,564.0 L248.3,564.0 L251.2,105.4 L254.1,564.0
 L257.0,564.0 L259.9,134.8 L262.8,529.1 L265.7,563.8 L268.6,563.7 L271.5,562.7 L274.4,563.0 L277.3,527.0
 L280.2,563.9 L283.1,563.9 L286.0,563.9 L288.9,524.4 L291.8,526.2 L294.7,563.9 L297.6,522.4 L300.5,516.1
 L303.4,563.9 L306.3,103.3 L309.2,119.1 L312.1,120.5 L315.0,545.6 L317.9,123.1 L320.8,543.4 L323.7,564.0
 L326.6,564.0 L329.5,564.0 L332.4,564.0 L335.3,133.0 L338.2,564.0 L341.1,564.0 L344.0,564.0 L346.9,135.0
 L349.8,113.5 L352.7,113.7 L355.6,114.5 L358.5,115.3 L361.4,539.2 L364.3,116.8 L367.2,537.6 L370.1,145.9
 L373.0,564.0 L375.9,147.3 L378.8,559.0 L381.7,151.7 L384.6,149.6 L387.5,538.3 L390.4,536.0 L393.3,151.8
 L396.2,546.9 L399.1,558.7 L402.0,159.5 L404.9,556.9 L407.8,156.2 L410.7,533.3 L413.6,517.5 L416.5,544.1
 L419.4,130.7 L422.3,527.4 L425.2,484.8 L428.1,520.1 L431.0,564.0 L433.9,182.6 L436.8,208.7 L439.7,513.5
 L442.6,556.3 L445.5,490.4 L448.4,135.6 L451.3,485.2 L454.2,135.9 L457.1,564.0 L460.0,493.9 L462.9,487.5
 L465.8,564.0 L468.7,537.1 L471.6,484.4 L474.5,135.1 L477.4,490.8 L480.3,545.4 L483.2,134.9 L486.1,485.2
 L489.0,134.5 L491.9,152.6 L494.8,193.1 L497.7,133.3 L500.6,133.5 L503.5,547.7 L506.4,133.6 L509.3,133.7
 L512.2,534.9 L515.1,546.1 L518.0,488.9 L520.9,547.2 L523.8,550.0 L526.7,178.8 L529.6,132.9 L532.5,556.7
 L535.4,148.1 L538.3,533.3 L541.2,147.3 L544.1,133.7 L547.0,135.0 L549.9,534.4 L552.8,135.1 L555.7,176.8
 L558.6,512.7 L561.5,158.3 L564.4,167.3 L567.3,135.3 L570.2,165.3 L573.1,153.6 L576.1,135.7 L579.0,135.0
 L581.9,153.9 L584.8,152.2 L587.7,136.1 L590.6,136.3 L593.5,563.9 L596.4,563.9 L599.3,136.0 L602.2,136.5
 L605.1,136.6 L608.0,137.3 L610.9,137.7 L613.8,161.6 L616.7,138.7 L619.6,139.2 L622.5,139.8 L625.4,140.2
 L628.3,140.8 L631.2,544.5 L634.1,163.0 L637.0,171.9 L639.9,143.5 L642.8,162.0 L645.7,144.4 L648.6,170.6
 L651.5,553.4 L654.4,177.0 L657.3,563.9 L660.2,144.3 '/>

In [ ]:
// foreach(var grp in pltTab1.dataGroups) {
//     Plot2Ddata dat = new Plot2Ddata();
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m2/" + pltTab1.Title + "/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

## m=4 cases (eta = {0.7, 0.6, 0.5, 0.3, 0.2})

In [ ]:
var pltTab3 = PlotTableDM[7,0];
PlotTableDM[8,0].dataGroups[0].Format.LineColor = LineColors.Magenta;
pltTab3 = pltTab3.Merge(PlotTableDM[8,0]);
PlotTableDM[9,0].dataGroups[0].Format.LineColor = LineColors.Green;
pltTab3 = pltTab3.Merge(PlotTableDM[9,0]);
PlotTableDM[10,0].dataGroups[0].Format.LineColor = LineColors.Orange;
pltTab3 = pltTab3.Merge(PlotTableDM[10,0]);
PlotTableDM[11,0].dataGroups[0].Format.LineColor = LineColors.Red;
pltTab3 = pltTab3.Merge(PlotTableDM[11,0]);

In [ ]:
pltTab3.XrangeMin = 0;
pltTab3.XrangeMax = 3;
var gp = pltTab3.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 1.6 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 BoSSS-J686m4Oh01eta07 
 
 
 
 
 BoSSS-J686m4 O h01 e ta07 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M455.3,57.1 L508.7,57.1 M53.9,114.1 L55.1,114.2 L56.2,114.1 L57.4,114.0 L58.5,113.8 L59.7,113.6
 L60.9,113.4 L62.0,113.2 L63.2,112.9 L64.3,112.8 L65.5,112.5 L66.7,112.0 L67.8,111.5 L69.0,111.0
 L70.1,110.8 L71.3,110.2 L72.5,109.6 L73.6,109.0 L74.8,109.0 L75.9,108.4 L77.1,107.8 L78.3,107.4
 L79.4,106.8 L80.6,106.2 L81.7,105.6 L82.9,104.8 L84.1,104.2 L85.2,103.7 L86.4,103.1 L87.5,102.5
 L88.7,101.9 L89.9,101.4 L91.0,100.6 L92.2,99.9 L93.4,99.1 L94.5,98.4 L95.7,96.0 L96.8,95.0
 L98.0,93.9 L99.2,93.2 L100.3,92.2 L101.5,90.5 L102.6,89.8 L103.8,88.5 L105.0,564.0 L106.1,85.7
 L107.3,83.9 L108.4,82.4 L109.6,81.1 L110.8,79.5 L111.9,78.1 L113.1,76.6 L114.2,75.1 L115.4,73.9
 L116.6,72.0 L117.7,70.8 L118.9,69.4 L120.0,67.6 L121.2,66.4 L122.4,64.8 L123.5,63.7 L124.7,62.5
 L125.8,61.3 L127.0,59.8 L128.2,58.6 L129.3,57.8 L130.5,56.9 L131.6,564.0 L132.8,564.0 L134.0,564.0
 L135.1,564.0 L136.3,564.0 L137.4,54.4 L138.6,54.6 L139.8,564.0 L140.9,564.0 L142.1,55.1 L143.2,56.3
 L144.4,57.6 L145.6,58.2 L146.7,564.0 L147.9,61.5 L149.0,64.5 L150.2,66.3 L151.4,69.3 L152.5,72.5
 L153.7,75.7 L154.8,79.1 L156.0,82.6 L157.2,86.3 L158.3,90.1 L159.5,94.1 L160.7,98.1 L161.8,102.9
 L163.0,107.5 L164.1,112.1 L165.3,117.0 L166.5,122.0 L167.6,127.2 L168.8,131.8 L169.9,135.2 L171.1,137.4
 L172.3,139.8 L173.4,142.4 L174.6,145.4 L175.7,148.1 L176.9,564.0 L178.1,564.0 L179.2,154.8 L180.4,157.0
 L181.5,563.7 L182.7,563.6 L183.9,563.6 L185.0,563.5 L186.2,563.5 L187.3,563.5 L188.5,563.5 L189.7,563.6
 L190.8,563.6 L192.0,563.6 L193.1,563.6 L194.3,564.0 L195.5,564.0 L196.6,563.9 L197.8,563.9 L198.9,143.3
 L200.1,141.6 L201.3,140.0 L202.4,138.4 L203.6,564.0 L204.7,564.0 L205.9,564.0 L207.1,131.2 L208.2,129.3
 L209.4,564.0 L210.5,125.6 L211.7,123.1 L212.9,123.4 L214.0,120.6 L215.2,564.0 L216.3,564.0 L217.5,564.0
 L218.7,563.9 L219.8,563.9 L221.0,563.8 L222.1,563.7 L223.3,112.7 L224.5,111.4 L225.6,111.2 L226.8,111.0
 L228.0,110.8 L229.1,563.9 L230.3,110.7 L231.4,563.8 L232.6,109.1 L233.8,109.4 L234.9,109.8 L236.1,109.3
 L237.2,109.9 L238.4,110.1 L239.6,110.8 L240.7,111.5 L241.9,112.0 L243.0,112.8 L244.2,112.4 L245.4,113.3
 L246.5,114.1 L247.7,114.0 L248.8,114.9 L250.0,114.4 L251.2,115.6 L252.3,115.9 L253.5,117.2 L254.6,117.6
 L255.8,118.5 L257.0,119.8 L258.1,120.9 L259.3,121.8 L260.4,122.6 L261.6,122.6 L262.8,123.9 L263.9,125.0
 L265.1,125.9 L266.2,126.1 L267.4,127.0 L268.6,128.3 L269.7,129.4 L270.9,130.4 L272.0,131.4 L273.2,132.4
 L274.4,132.8 L275.5,133.8 L276.7,134.8 L277.8,135.9 L279.0,136.9 L280.2,137.8 L281.3,138.7 L282.5,139.6
 L283.6,140.5 L284.8,141.4 L286.0,142.3 L287.1,143.3 L288.3,144.2 L289.4,145.1 L290.6,146.0 L291.8,147.0
 L292.9,147.9 L294.1,148.9 L295.2,149.8 L296.4,150.8 L297.6,152.6 L298.7,153.9 L299.9,154.9 L301.1,155.9
 L302.2,156.8 L303.4,157.7 L304.5,158.7 L305.7,159.6 L306.9,160.6 L308.0,161.7 L309.2,162.8 L310.3,163.9
 L311.5,165.1 L312.7,166.3 L313.8,167.5 L315.0,168.8 L316.1,170.0 L317.3,171.2 L318.5,172.5 L319.6,173.7
 L320.8,174.9 L321.9,176.8 L323.1,178.2 L324.3,179.3 L325.4,180.6 L326.6,181.9 L327.7,183.3 L328.9,185.0
 L330.1,186.6 L331.2,187.8 L332.4,189.2 L333.5,190.6 L334.7,193.1 L335.9,194.2 L337.0,564.0 L338.2,198.0
 L339.3,199.4 L340.5,201.0 L341.7,202.6 L342.8,562.2 L344.0,206.0 L345.1,208.6 L346.

In [ ]:
// foreach(var grp in pltTab1.dataGroups) {
//     Plot2Ddata dat = new Plot2Ddata();
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m2/" + pltTab1.Title + "/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }